In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

df = pd.read_csv('cdc_data/states_mcd_per_year.csv')

causes = df.pivot_table(values='deaths', index=['state', 'state-code', 'year', 'population'],
                        columns='multiple-cause-of-death', fill_value=0, aggfunc='max').reset_index()

causes.columns = ['state','state_code','year','population','heroin_deaths','methadone_deaths',
                  'other_opioids_deaths','other_synthetics_deaths']

rate = df.pivot_table(values='crude-rate', index=['state', 'year'],columns='multiple-cause-of-death',
                      fill_value=0, aggfunc='max').reset_index()

rate.columns=['state','year','heroin_death_rate','methadone_death_rate','other_opioids_death_rate',
              'other_synthetics_death_rate']

states_year = pd.merge(causes,rate,how='inner',on=['state','year'])

states_year = states_year[['state_code','state','year','heroin_deaths','other_opioids_deaths',
                           'methadone_deaths','other_synthetics_deaths','population',
                           'heroin_death_rate','other_opioids_death_rate','methadone_death_rate',
                           'other_synthetics_death_rate']]

c_df = pd.read_csv('cdc_data/counties_mcd_2006-2017.csv')

c_causes = c_df.pivot_table(values='deaths', index=['county-code', 'county', 'state', 'population'],
                            columns='multiple-cause-of-death', fill_value=0, aggfunc='max').reset_index()

c_causes.columns = ['county_code','county','state','population','heroin_deaths','methadone_deaths',
                    'other_opioids_deaths','other_synthetics_deaths']

c_rate = c_df.pivot_table(values='crude-rate', index=['county-code','county','state'],
                          columns='multiple-cause-of-death',fill_value=0, aggfunc='max').reset_index()

c_rate.columns=['county_code','county','state','heroin_death_rate','methadone_death_rate',
                'other_opioids_death_rate','other_synthetics_death_rate']

counties_range = pd.merge(c_causes,c_rate,how='inner',on=['county_code','county','state'])

counties_range = counties_range[['county_code','county','state','heroin_deaths','other_opioids_deaths',
                                 'methadone_deaths','other_synthetics_deaths','population',
                                 'heroin_death_rate','other_opioids_death_rate','methadone_death_rate',
                                 'other_synthetics_death_rate']]

def state_prescribing_rate():
    years = list(range(2006,2018))

    tables = []

    for year in years:
        url = f'https://www.cdc.gov/drugoverdose/maps/rxstate{year}.html'

        table = pd.read_html(url)[0]

        table.columns = ['state','abbr',f'prescribing_rate_{year}']

        tables.append(table)
        
    states_prescribe_df = pd.merge(tables[0],tables[1],how='inner',on=['state','abbr'])

    for x in range(len(tables)):
        try:
            states_prescribe_df = pd.merge(states_prescribe_df,tables[x+2],how='inner',on=['state','abbr'])

        except:
            break

    states_prescribe_df = states_prescribe_df.drop(columns='abbr')
    
    states_prescribe_df = states_prescribe_df.T.reset_index()

    for x in range(len(states_prescribe_df)):
        try:
            states_prescribe_df['index'].loc[x+1] = years[x]
        except:
            break
            
    states_prescribe_df.columns = states_prescribe_df.loc[0]

    states_prescribe_df = states_prescribe_df.iloc[1:]\
                                                .rename(columns={'state':'year'})\
                                                .set_index('year').unstack()\
                                                .to_frame(name='states_prescribe_df')\
                                                .reset_index()

    states_prescribe_df.columns = ['state','year','prescribing_rate']
    
    return states_prescribe_df

states_prescribe_df = state_prescribing_rate()

states_each_year = pd.merge(states_year,states_prescribe_df,how='outer',on=['state','year'])\
                        .replace(['Suppressed','Unreliable'],np.NaN)

def county_prescribing_rate():
    years = list(range(2006,2017))

    tables = []

    for year in years:
        url = f'https://www.cdc.gov/drugoverdose/maps/rxcounty{year}.html'

        table = pd.read_html(url)[0]

        table.columns = ['county','state','county_code',f'prescribing_rate_{year}']

        split = table['county'].str.split(', ',n=1,expand=True)

        table['county'] = split[0]

        tables.append(table)
        
    counties_prescribe_df = pd.merge(tables[0],tables[1],how='inner',on=['county','state','county_code'])

    for x in range(len(tables)):
        try:
            counties_prescribe_df = pd.merge(counties_prescribe_df,tables[x+2],how='inner',
                                             on=['county','state','county_code'])

        except:
            break
            
    url_2017 = f'https://www.cdc.gov/drugoverdose/maps/rxcounty2017.html'

    table_2017 = pd.read_html(url_2017)[0]

    table_2017.columns = ['county','state','county_code','prescribing_rate_2017']
    
    table_2017 = table_2017.drop(columns=['county','state'])

    counties_prescribe_df = pd.merge(counties_prescribe_df,table_2017,how='inner',on='county_code')

    counties_prescribe_df = counties_prescribe_df.drop(columns='state')
    
    return counties_prescribe_df

counties_prescribe_df = county_prescribing_rate()

counties_all_years = pd.merge(counties_range,counties_prescribe_df,how='inner',on='county_code')\
                        .rename(columns={'county_x':'county'})\
                        .drop(columns='county_y')\
                        .replace(['Suppressed','Unreliable'],np.NaN)

states_each_year.to_csv('states_each_year.csv',index=False)

counties_all_years.to_csv('counties_all_years.csv',index=False)

In [2]:
states_each_year.head()

,state_code,state,year,heroin_deaths,other_opioids_deaths,methadone_deaths,other_synthetics_deaths,population,heroin_death_rate,other_opioids_death_rate,methadone_death_rate,other_synthetics_death_rate,prescribing_rate
0,1,Alabama,1999,NaN,10,16,10,4430141,NaN,NaN,NaN,NaN,NaN
1,1,Alabama,2000,NaN,26,12,11,4447100,NaN,0.6,NaN,NaN,NaN
2,1,Alabama,2001,NaN,22,26,NaN,4467634,NaN,0.5,0.6,NaN,NaN
3,1,Alabama,2002,NaN,30,33,NaN,4480089,NaN,0.7,0.7,NaN,NaN
4,1,Alabama,2003,NaN,22,25,NaN,4503491,NaN,0.5,0.6,NaN,NaN


In [3]:
counties_all_years.head()

,county_code,county,state,heroin_deaths,other_opioids_deaths,methadone_deaths,other_synthetics_deaths,population,heroin_death_rate,other_opioids_death_rate,...,prescribing_rate_2008,prescribing_rate_2009,prescribing_rate_2010,prescribing_rate_2011,prescribing_rate_2012,prescribing_rate_2013,prescribing_rate_2014,prescribing_rate_2015,prescribing_rate_2016,prescribing_rate_2017
0,1001,Autauga County,Alabama,NaN,NaN,NaN,NaN,653405,NaN,NaN,...,144.9,147.5,151.7,144.1,157.8,166.7,145.3,129.9,129.6,106.6
1,1003,Baldwin County,Alabama,NaN,75,61,37,2276081,NaN,3.3,...,138.3,143.9,143.8,150.0,156.4,154.3,143.5,132.1,123.8,106.7
2,1005,Barbour County,Alabama,NaN,NaN,NaN,NaN,324547,NaN,NaN,...,79.6,88.5,97.0,99.4,118.3,107.5,102.0,93.3,92.7,90.7
3,1007,Bibb County,Alabama,NaN,NaN,NaN,NaN,271373,NaN,NaN,...,119.6,109.6,58.9,57.7,69.2,70.5,75.8,69.4,97.2,80.6
4,1009,Blount County,Alabama,15,10,17,13,687927,NaN,NaN,...,58.2,56.2,60.1,64.8,66.6,65.9,63.2,57.9,56.9,48.9


In [4]:
state_engine = create_engine('sqlite:///states_each_year.sqlite')
state_conn = state_engine.connect()

Base = declarative_base()

class State(Base):
    __tablename__ = 'states_each_year'
    state_code = Column(Integer, primary_key=True)
    state = Column(String, primary_key=True)
    year = Column(Integer, primary_key=True)
    heroin_deaths = Column(Integer)
    other_opioids_deaths = Column(Integer)
    methadone_deaths = Column(Integer)
    other_synthetics_deaths = Column(Integer)
    population = Column(Integer)
    heroin_death_rate = Column(Float)
    other_opioids_death_rate = Column(Float)
    methadone_death_rate = Column(Float)
    other_synthetics_death_rate = Column(Float)
    prescribing_rate = Column(Float)
    
Base.metadata.create_all(state_conn)

states_each_year.to_sql(State.__tablename__, state_conn, index=False, if_exists='replace')

county_engine = create_engine('sqlite:///counties_all_years.sqlite')
county_conn = county_engine.connect()

Base = declarative_base()

class County(Base):
    __tablename__ = 'counties_all_years'
    county_code = Column(Integer, primary_key=True)
    county = Column(String)
    state = Column(String)
    heroin_deaths = Column(Integer)
    other_opioids_deaths = Column(Integer)
    methadone_deaths = Column(Integer)
    other_synthetics_deaths = Column(Integer)
    population = Column(Integer)
    heroin_death_rate = Column(Float)
    other_opioids_death_rate = Column(Float)
    methadone_death_rate = Column(Float)
    other_synthetics_death_rate = Column(Float)
    prescribing_rate_2006 = Column(Float)
    prescribing_rate_2007 = Column(Float)
    prescribing_rate_2008 = Column(Float)
    prescribing_rate_2009 = Column(Float)
    prescribing_rate_2010 = Column(Float)
    prescribing_rate_2011 = Column(Float)
    prescribing_rate_2012 = Column(Float)
    prescribing_rate_2013 = Column(Float)
    prescribing_rate_2014 = Column(Float)
    prescribing_rate_2015 = Column(Float)
    prescribing_rate_2016 = Column(Float)
    prescribing_rate_2017 = Column(Float)
    
Base.metadata.create_all(county_conn)

counties_all_years.to_sql(County.__tablename__, county_conn, index=False, if_exists='replace')